In [ ]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')

categories = {"Product": [], "Quantity": [], "Purchase_price": [],"Selling_price": [],"Gross_revenue": [],"Net_revenue": []}

Mounted at /content/drive


In [35]:
#English Version


def help():
    """
    this function display commands when user type help
    """
    answers = ["s", "sì", "si", "yes", "y"]
    request = input("Hai bisogno di aiuto? [Sì/No]: ")

    if request.lower().strip() in answers:
        # Definisco i codici per pulizia
        BOLD = "\033[1m"
        RESET = "\033[0m"

        print(f"\nI comandi disponibili sono i seguenti:\n"
              f"{BOLD}aggiungi:{RESET} aggiungi un prodotto al magazzino\n"
              f"{BOLD}elenca:{RESET} elenca i prodotti presenti nel magazzino\n"
              f"{BOLD}vendita:{RESET} registra una vendita\n"
              f"{BOLD}profitti:{RESET} mostra i profitti totali\n"
              f"{BOLD}chiudi:{RESET} esci dal programma")
    else:
        print("see you later!")


def add():
    """
    Aggiunge prodotti e relativi prezzi, se il prodotto è già presente,
    la quantità va a sommarsi a quella precedente.
    """
    product = input("Nome prodotto: ").lower().strip()

    quantity = None
    purchase_price = None
    selling_price = None

    #Controlla se il prodotto è già presente nel catalogo
    if product in categories["Product"]:
        #Se presente, chiede di inserire solo la quantità da aggiungere
        while quantity is None or quantity<=0:
            try:
                quantity = float(input("Quantità: "))
                if quantity<=0:
                    print("devi inserire una quantità maggiore di zero")
            except ValueError:
                print("hai inserito una quantità non valida")

        index = categories["Product"].index(product)
        categories["Quantity"][index] += quantity
    else:
        #Se il prodotto è nuovo, chiede di inserire tutte le informazioni
        while quantity is None or quantity<=0:
            try:
                quantity = float(input("Quantità: "))
                if quantity<=0:
                    print("devi inserire una quantità maggiore di zero")
            except ValueError:
                print("hai inserito una quantità non valida")
        while purchase_price is None:
            try:
                purchase_price = float(input("Prezzo di acquisto per singolo prodotto: "))
            except ValueError:
                print("hai inserito un prezzo non valido")
        while selling_price is None:
            try:
                selling_price = float(input("Prezzo di vendita: "))
            except ValueError:
                print("hai inserito un prezzo non valido")

        categories["Product"].append(product)
        categories["Quantity"].append(quantity)
        categories["Purchase_price"].append(purchase_price)
        categories["Selling_price"].append(selling_price)

    print(f"AGGIUNTO: {product} x {quantity}")

    file_path = "/content/drive/MyDrive/DATA ANALISI/PROFESSION AI/6 Python/new_catalog.txt"
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, "w") as catalog:
        catalog.write("Product          Quantity         Purchase Price        Selling Price\n")
        for i in range(len(categories["Product"])):
            catalog.write(f'{categories["Product"][i]:<17} {str(categories["Quantity"][i]):<17} {str(categories["Purchase_price"][i]):<20} {str(categories["Selling_price"][i])}\n')


def sale():
    """
    this function records a sale of a product
    """
    receipt = [[], [], [], []]  # Questa lista serve per stampare lo "Scontrino"
    # Nella sottolista 0 si aggiungono i prodotti da vendere
    # Nella sottolista 1 si aggiunge la quantità da vendere
    # Nella sottolista 2 si aggiungono i prezzi di vendita dei prodotti da vendere
    # Nella sottolista 3 si aggiungono i prezzi di acquisto dei prodotti da vendere

    total = 0  # Inizializzazione della variabile totale che andrà stampata nello scontrino finale

    while True:
        product = input("Product name: ").lower().strip()
        quantity = None

        # SE IL PRODOTTO è PRESENTE
        if product in categories["Product"]:
            index = categories["Product"].index(product)

            while quantity is None or quantity <= 0:
                try:
                    quantity = float(input("Quantità: "))
                    if quantity <= 0:
                        print("Devi inserire una quantità maggiore di zero.")
                except ValueError:
                    print("Hai inserito una quantità non valida.")

            if categories["Quantity"][index] >= quantity:
                categories["Quantity"][index] -= quantity
                receipt[0].append(product)
                receipt[1].append(quantity)
                receipt[2].append(categories["Selling_price"][index])
                receipt[3].append(categories["Purchase_price"][index])
                print("Sale registered")
            else:
                print("Risorse insufficienti per soddisfare la richiesta.")
        else:
            print("Prodotto non presente nel magazzino.")

        command2 = input("Aggiungere un altro prodotto? [Sì/No]: ").lower().strip()
        if command2 not in ["si", "sì"]:
            break

    print("VENDITA REGISTRATA:")
    for i in range(len(receipt[0])):
        price = receipt[2][i]
        revenue = price * receipt[1][i]
        total += revenue
        print(f"{receipt[1][i]} X {receipt[0][i]}: {price:.2f} €")

    income = sum(receipt[1][i] * receipt[3][i] for i in range(len(receipt[0])))
    net_revenue = total - income
    categories.setdefault("Gross_revenue", []).append(total)
    categories.setdefault("Net_revenue", []).append(net_revenue)

    print(f"TOTALE: {total:.2f} €")
    print(f"RICAVO NETTO: {net_revenue:.2f} €")

    file_path = "/content/drive/MyDrive/DATA ANALISI/PROFESSION AI/6 Python/new_catalog.txt"
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, "w") as catalog:
        catalog.write("Product          Quantity         Purchase Price        Selling Price\n")
        for i in range(len(categories["Product"])):
            catalog.write(f'{categories["Product"][i]:<17} {str(categories["Quantity"][i]):<17} {str(categories["Purchase_price"][i]):<20} {str(categories["Selling_price"][i])}\n')


def profits():
    """
    this function dispay a the gross revenue e and net revenue
    """
    gross_profitt_total=sum(categories['Gross_revenue'])
    net_profitt_total=sum(categories['Net_revenue'])
    print(f"Profitto lordo: {gross_profitt_total} €")
    print(f"Profitto netto: {net_profitt_total} €")


def list_products():
    """
    this function give in response the list of products quantities and prices of all
    """
    print("\nProduct       Quantity       Price")
    for i in range(len(categories["Product"])):
        print(f"{categories['Product'][i]:<15} {categories['Quantity'][i]:<15} {categories['Selling_price'][i]}€")



def reset():
    comando = input("Vuoi resettare tutta la cassa? [Sì/No] \n(Rispondendo Sì non viene cancellato anche il file salvato): ").strip().lower()
    risposta = ["sì", "si"]  # ⬅️ Meglio minuscolo perché usi .lower()

    if comando in risposta:
        # Svuota tutte le liste del dizionario
        categories["Product"].clear()
        categories["Quantity"].clear()
        categories["Purchase_price"].clear()
        categories["Selling_price"].clear()
        categories["Gross_revenue"].clear()
        categories["Net_revenue"].clear()

        cancella_tutto = input("Vuoi cancellare anche il file salvato? [Sì/No]: ").strip().lower()

        if cancella_tutto in risposta:
            try:
                import os
                percorso = "/content/drive/MyDrive/DATA ANALISI/PROFESSION AI/6 Python/new_catalog.txt"
                if os.path.exists(percorso):
                    os.remove(percorso)  # ⬅️ USA LO STESSO PERCORSO!
                    print("File catalogo eliminato")
                else:
                    print("File non trovato")
            except Exception as e:
                print(f"Errore: {e}")

        print("CATALOGO COMPLETAMENTE RESETTATO")
    else:
        print("Operazione annullata")


def command():
    while True:
        command_input = input("\nInserisci un comando: \nSe non sai che comando inserire digita 'aiuto' \n"
                             "per interrompere programma digitare 'chiudi': ").lower().strip()
        if command_input == "chiudi":
            print("Arrivederci!")
            break
        elif command_input == "aiuto":
            help()
        elif command_input == "aggiungi":
            add()
        elif command_input == "vendita":
            sale()
        elif command_input == "profitti":
            profits()
        elif command_input == "elenca":
            list_products()
        elif command_input == "reset":
            reset()
        else:
            return_invalid()


command()


Inserisci un comando: 
Se non sai che comando inserire digita 'aiuto' 
per interrompere programma digitare 'chiudi': clkcl
Invalid command
The available commands are as follows:
aggiungi: aggiungi un prodotto al magazzino
elenca: elenca i prodotti presenti nel magazzino
vendita: registra una vendita
profitti: mostra i profitti totali
aiuto: mostra i comandi disponibili
chiudi: esci dal programma


KeyboardInterrupt: Interrupted by user